In [1]:
import numpy as np
import tensorflow as tf

2024-02-02 18:05:05.382921: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 18:05:05.403004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 18:05:05.403025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 18:05:05.403521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 18:05:05.406915: I tensorflow/core/platform/cpu_feature_guar

## Load and Test 196_25_10 DNN Model

In [18]:
input_size = 14 * 14
layer_1 = 25
n_class = 10

inputs = tf.keras.layers.Input(shape=(input_size,))
out = tf.keras.layers.Dense(layer_1)(inputs)
out = tf.keras.layers.ReLU()(out) # not support dor dense layer
out = tf.keras.layers.Dense(n_class)(out)
#out = Softmax()(out)
model = tf.keras.Model(inputs, out)

In [9]:
model_path = "../../models/"
arch_folder = "input-dense-dense/"
model_name = "c_aware_196_25_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense_2 (Dense)             (None, 25)                4925      
                                                                 
 re_lu_1 (ReLU)              (None, 25)                0         
                                                                 
 dense_3 (Dense)             (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

In [11]:
# Resize for 14 * 14 images
train_images_tf_reshaped = tf.reshape(train_images_tf, [-1, 28, 28, 1])  # Reshape to [num_samples, height, width, channels]
test_images_tf_reshaped = tf.reshape(test_images_tf, [-1, 28, 28, 1])

# Downsample images
train_images_tf_downsampled = tf.image.resize(train_images_tf_reshaped, [14, 14], method='bilinear')
test_images_tf_downsampled = tf.image.resize(test_images_tf_reshaped, [14, 14], method='bilinear')

# Flatten the images back to [num_samples, 14*14]
train_images_tf = tf.reshape(train_images_tf_downsampled, [-1, 14*14])
test_images_tf = tf.reshape(test_images_tf_downsampled, [-1, 14*14])

In [12]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.8117 - accuracy: 0.9541 - 350ms/epoch - 1ms/step

Test accuracy: 0.9541000127792358


### Setup

In [15]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [44]:
def transfer_weights(layers, model, scalar = 36):
    weights = []
    biases = []
    for ind in range(len(layers)-1):
        w = [[int(model.weights[ind * 2].numpy()[i][j]*10**scalar) for j in range(layers[ind+1])] for i in range(layers[ind])]
        b = [int(model.weights[ind * 2 + 1].numpy()[i]*10**(scalar * 2)) for i in range(layers[ind+1])]
        
        weights.append(w)
        biases.append(b)

    return weights, biases


In [47]:
weights, biases = transfer_weights([196,25,10], model)

In [72]:
weights[1][0]

[-124483436346054082331709472319209472,
 -266745209693908690992144174381268992,
 125370115041732800785437157077549056,
 119624227285385135013574016348717056,
 -385843664407730101924536114693013504,
 -360206186771392871887340280444092416,
 222950249910354639194961063352205312,
 61239928007125854950088778125934592,
 166527777910232544193034274289483776,
 408954054117202740136896037186961408]

In [299]:
def relu_mod(x):
    return x if x < p // 2 else 0

def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    #print (len(input), nInputs)
    
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]

        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p
        
    return Input, Weights, Bias, out, remainder

In [339]:
def DenseInt_(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]

    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
        
    return Input, Weights, Bias, out, remainder

In [392]:
def prepare_input_json(layers, weights, biases, x_in, scalar=36, relu = False):
    relu_outs = []
    dense_weights = []
    dense_biases = []
    dense_outs = []
    dense_remainders = []
    x_ins = []

    out = x_in
    for ind in range(len(weights)):
        nInputs = layers[ind]
        nOutputs = layers[ind + 1]
        #print (nInputs, nOutputs)
        x_in, w, b, out, rem = DenseInt(nInputs, nOutputs, 10 ** scalar, 
                                     out, weights[ind], biases[ind])
        
        dense_outs.append(out)
        if relu:
            out = [x if x < p//2 else 0 for x in out]
            relu_outs.append([str(x) for x in out])

        #print (out)
        dense_weights.append(w)
        dense_biases.append(b)
        
        dense_remainders.append(rem)
        x_ins.append(x_in)

    dense_outs = [[str(x) for x in sub] for sub in dense_outs]
        
    return x_ins[0], dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs

#### Test on simple model

In [393]:
layers = [784, 56, 10]
inputs = tf.keras.layers.Input(shape=(784,))
out = tf.keras.layers.Dense(56)(inputs)
out = tf.keras.layers.ReLU()(out) # not support dor dense layer
out = tf.keras.layers.Dense(10)(out)
#out = Softmax()(out)

simple_model = tf.keras.Model(inputs, out)

simple_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

simple_model.summary()

Model: "model_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 784)]             0         
                                                                 
 dense_67 (Dense)            (None, 56)                43960     
                                                                 
 re_lu_23 (ReLU)             (None, 56)                0         
                                                                 
 dense_68 (Dense)            (None, 10)                570       
                                                                 
Total params: 44530 (173.95 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [394]:
X = np.random.rand(1,784)

X_in = [int(x*1e36) for x in X[0]]

In [395]:
weights, biases = transfer_weights(layers, simple_model)

In [396]:
x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs = prepare_input_json(layers, weights, biases, X_in, relu=True)
dense_outs[-1]

['21888242871839275222246405745257275088547712741010285124653134685975492047989',
 '64461780081078897374926454874303670',
 '206099086914864141015720331481235453',
 '301288055733862308976226566011151968',
 '21888242871839275222246405745257275088548082637687730584578687394167161939996',
 '21872647421375424758135125470329348',
 '74987286570992543954090709101517465',
 '21888242871839275222246405745257275088547587655272012960865672758172769230540',
 '1111638609321312424540851124796407115',
 '1855937309942566737994135755662048055']

In [397]:
X_in, Dense32weights, Dense32bias, Dense32out, Dense32remainder = DenseInt_(784,56, 10**36, X_in, weights[0], biases[0])

In [398]:
ReLUout = [Dense32out[i] if int(Dense32out[i]) < p//2 else 0 for i in range(56)]
Dense21in = [int(ReLUout[i]) for i in range(56)]

In [399]:
_, Dense21weights, Dense21bias, Dense21out, Dense21remainder = DenseInt_(56, 10, 10**36, Dense21in, weights[1], biases[1])
Dense21out

['21888242871839275222246405745257275088547712741010285124653134685975492047989',
 '64461780081078897374926454874303670',
 '206099086914864141015720331481235453',
 '301288055733862308976226566011151968',
 '21888242871839275222246405745257275088548082637687730584578687394167161939996',
 '21872647421375424758135125470329348',
 '74987286570992543954090709101517465',
 '21888242871839275222246405745257275088547587655272012960865672758172769230540',
 '1111638609321312424540851124796407115',
 '1855937309942566737994135755662048055']

In [400]:
simple_model.predict(X)

1/1 [==============================] - 0s 33ms/step


array([[-0.6516593 ,  0.06446189,  0.2060991 ,  0.30128804, -0.28176275,
         0.02187276,  0.07498729, -0.7767452 ,  1.1116385 ,  1.8559372 ]],
      dtype=float32)

In [401]:
import json

in_json = {
    "in": x_in,
    "Dense32weights": dense_weights[0],
    "Dense32bias": dense_biases[0],
    "Dense32out": dense_outs[0],
    "Dense32remainder": dense_remainders[0],
    "ReLUout": relu_outs[0], 
    "Dense21weights": dense_weights[1],
    "Dense21bias": dense_biases[1],
    "Dense21out": dense_outs[1],
    "Dense21remainder": dense_remainders[1]
}


with open("./tmp/input.json", "w") as f:
    json.dump(in_json, f)

In [402]:
ReLUout

[0,
 '1740378166542656759482186281727063207',
 0,
 '665993099727507384065029260156218697',
 0,
 '69577348329897550250554733933848429',
 0,
 '1476298976826797441633872004018772249',
 0,
 0,
 '944210392939385517650387649480375792',
 '356518878668041190610455646138776409',
 '566602428580131055096567038447823438',
 '1097513366205503365421702612922238755',
 '835816972746304142909475253183466632',
 0,
 0,
 '1099273191099292480449694720932948965',
 0,
 '144493945312701031958715819998514811',
 0,
 '160947886835453079542876893767116428',
 0,
 0,
 '739017511639276095775829576274234224',
 0,
 0,
 0,
 0,
 0,
 '757892442836530559228984227040716457',
 '415601943532006337034280277131910761',
 '12310543291558788459229224797328546',
 0,
 0,
 0,
 '452743524094830236747848819391631502',
 0,
 '1293160281003872094798548686074372589',
 0,
 0,
 '115641193966956731293473882435038396',
 0,
 '729732199508907024481784595377895346',
 0,
 '127914070154402518361126973111992041',
 0,
 0,
 0,
 0,
 '977306076171245541

In [403]:
relu_outs[0]

['0',
 '1740378166542656759482186281727063207',
 '0',
 '665993099727507384065029260156218697',
 '0',
 '69577348329897550250554733933848429',
 '0',
 '1476298976826797441633872004018772249',
 '0',
 '0',
 '944210392939385517650387649480375792',
 '356518878668041190610455646138776409',
 '566602428580131055096567038447823438',
 '1097513366205503365421702612922238755',
 '835816972746304142909475253183466632',
 '0',
 '0',
 '1099273191099292480449694720932948965',
 '0',
 '144493945312701031958715819998514811',
 '0',
 '160947886835453079542876893767116428',
 '0',
 '0',
 '739017511639276095775829576274234224',
 '0',
 '0',
 '0',
 '0',
 '0',
 '757892442836530559228984227040716457',
 '415601943532006337034280277131910761',
 '12310543291558788459229224797328546',
 '0',
 '0',
 '0',
 '452743524094830236747848819391631502',
 '0',
 '1293160281003872094798548686074372589',
 '0',
 '0',
 '115641193966956731293473882435038396',
 '0',
 '729732199508907024481784595377895346',
 '0',
 '1279140701544025183611269

In [269]:
simple_model.predict(X)

1/1 [==============================] - 0s 23ms/step


array([[-0.201047  ,  0.84194046, -0.3172891 ,  1.0736947 , -1.0347209 ,
        -0.389936  ,  0.96934855,  0.91301656,  0.27011284, -0.561428  ]],
      dtype=float32)

In [270]:
import psutil, time, threading, subprocess
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    print(f"Maximum memory used: {max_memory} MB")
    return max_memory

def run_bash(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Get the process ID
    pid = process.pid
    print(f"Process ID: {pid}")

    # Start memory monitoring in a separate thread
    monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
    monitor_thread.start()

    # Wait for the process to complete and capture output
    stdout, stderr = process.communicate()

    # Wait for the monitoring thread to finish
    monitor_thread.join()
    return stdout, stderr

In [324]:
import os
circuit_folder = "./golden_circuits/"
target_circom = "784_56_10.circom"
target_folder = "./tmp/"
os.makedirs(target_folder, exist_ok=True)

command = ['circom', circuit_folder + target_circom, "--r1cs", "--wasm", "--sym", "-o", target_folder]

In [294]:
subprocess.run(command)

template instances: 15
non-linear constraints: 73416
linear constraints: 0
public inputs: 0
private inputs: 45502 (45380 belong to witness)
public outputs: 10
wires: 118629
labels: 460243
Written successfully: ./tmp/784_56_10.r1cs
Written successfully: ./tmp/784_56_10.sym
Written successfully: ./tmp/784_56_10_js/784_56_10.wasm
Everything went okay


CompletedProcess(args=['circom', './golden_circuits/784_56_10.circom', '--r1cs', '--wasm', '--sym', '-o', './tmp'], returncode=0)

In [404]:
json_folder = target_folder + target_circom[:-7] + "_js/"
wit_file = json_folder + "generate_witness.js"
wasm_file = json_folder + target_circom[:-7] + ".wasm"
input_path = target_folder + "input.json"

command = ['node', wit_file, wasm_file, input_path, target_folder + "witness.wtns" ]

In [405]:
command

['node',
 './tmp/784_56_10_js/generate_witness.js',
 './tmp/784_56_10_js/784_56_10.wasm',
 './tmp/input.json',
 './tmp/witness.wtns']

In [406]:
subprocess.run(command)

CompletedProcess(args=['node', './tmp/784_56_10_js/generate_witness.js', './tmp/784_56_10_js/784_56_10.wasm', './tmp/input.json', './tmp/witness.wtns'], returncode=0)

1/1 [==============================] - 0s 33ms/step


array([[ 0.05200598,  0.18067959, -1.1555531 , -0.4750196 , -0.50847775,
         0.6045419 , -1.0021639 ,  0.38748404,  0.20723125, -0.13139105]],
      dtype=float32)

In [5]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

2024-02-01 04:44:02.687455: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 04:44:02.720314: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 04:44:02.720339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 04:44:02.721000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 04:44:02.725827: I tensorflow/core/platform/cpu_feature_guar

In [6]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

In [45]:
num_classes = 10
model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28*28,)),  # Adjusted for 28x28 images
    #keras.layers.Dense(128, activation='relu'),     # Increased number of neurons
    keras.layers.Dense(56),      # Additional hidden layer
    keras.layers.Dense(num_classes)  # Output layer for 10 classes
])

# model_tf.compile(optimizer='adam', 
#               loss='sparse_categorical_crossentropy', 
#               metrics=['accuracy'])

# model_tf.summary()

In [46]:
model_tf = tf.keras.models.load_model(model_path+arch_folder+model_name)

In [47]:
# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.3146 - accuracy: 0.9234 - 217ms/epoch - 694us/step

Test accuracy: 0.9233999848365784


## Scaling

In [48]:
import numpy as np

# Assuming test_images_tf is your flattened and normalized test dataset
scaled_test_images_tf = np.array([int(x * 1e36) for x in test_images_tf.flatten()]).reshape(test_images_tf.shape)

### Test

In [49]:
# Scale the weights and biases
weights = []
biases= []

for layer in model_tf.layers:
    #print ('new layer')
    weight, bias = layer.get_weights()
    scaled_weights = weight * 1e36
    scaled_biases = bias * 1e72
    weights.append(scaled_weights)
    biases.append(scaled_biases)
    

In [50]:
# using relu for dense layer is not supported by keras2circom
def relu_mod(x, p):
    return x if x < p // 2 else 0

In [51]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias, act = False):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]

    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += int(input[i] * weights[i][j])
        out[j] += int(bias[j])

        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p

    if act:
        out = [int(relu_mod(out[i], p)) for i in range(nOutputs)]

    return Input, Weights, Bias, out, remainder    

In [256]:
ind = 6
X = test_images_tf[ind:ind+1]
print (X.shape)
pred = model_tf.predict(X)
X_in = scaled_test_images_tf[ind]

np.argmax(pred, axis = 1)

(1, 784)
1/1 [==============================] - 0s 11ms/step


array([4])

In [158]:
from tensorflow import keras

# Assuming model_tf is your original model
# Extract the output of the second-to-last layer (the last dense layer before softmax)
layer_output = model_tf.layers[0].output

# Create a new model that gives the output of the last dense layer
intermediate_model = keras.Model(inputs=model_tf.input, outputs=layer_output)

intermediate_model.predict(X.reshape(1, -1))  # Reshape if necessary

# 'output' now contains the output of the last dense layer before softmax

1/1 [==============================] - 0s 18ms/step


array([[-9.5329070e-01,  1.4292629e+00, -8.4842488e-02,  2.8307574e+00,
        -2.3221037e+00,  7.0742428e-02,  5.5277711e-01,  6.6419549e-02,
        -1.0973787e-01,  1.1209676e+00, -3.1638861e-01, -1.3844252e+00,
        -5.3046063e-02, -1.2403058e+00, -1.1619565e+00, -7.9521552e-02,
        -3.3932751e-01,  8.1592131e-01, -3.9168000e-01,  4.8189560e-01,
        -6.7942822e-01,  1.8367920e+00,  1.0939014e-01,  5.4528899e+00,
        -2.8391507e-01,  2.5883527e+00, -1.7390031e-01, -2.0343879e-01,
         8.3122027e-01,  2.7671874e-01, -1.0484387e+00, -1.5784721e-01,
        -1.2740493e+00, -7.4017096e-01, -6.7795044e-01,  2.8966603e+00,
        -1.0565984e+00,  6.9625092e-01, -7.3894083e-01, -5.9243518e-01,
        -1.1391294e-01, -6.9581002e-02,  6.0503864e-01,  1.1942644e+00,
        -2.2848437e+00,  6.9404769e-01,  2.3846364e-01, -1.1942775e+00,
        -1.7259982e-01, -1.9012207e-01, -3.2332522e-01, -1.1359926e+00,
         3.0215591e-02, -1.9402095e+00, -2.0712204e-01, -3.39972

In [159]:
_, _, _, dense_out_1, remainder = DenseInt(784, 56, 10**36, X_in, weights[0], biases[0], act=False)
dense_out_1

[21888242871839275222246405745257275088547411109553182693019696267136279654625,
 1429262742189608892055161120894574842,
 21888242871839275222246405745257275088548279557790178464889343021438897624709,
 2830757570204967967106890586558348539,
 21888242871839275222246405745257275088546042296642297433340719353032481935672,
 70742345249902255278650529196150687,
 552776922231622311446666213875167399,
 66419575272351726118602312065397687,
 21888242871839275222246405745257275088548254662601433364243537257700439099999,
 1120967573676751623773976896854759091,
 21888242871839275222246405745257275088548048011794087370785156376648925069369,
 21888242871839275222246405745257275088546979975406822561717231158235568905148,
 21888242871839275222246405745257275088548311354453930159128493724373476611933,
 21888242871839275222246405745257275088547124094657919326189376498548964114087,
 21888242871839275222246405745257275088547202443968568112703218216218499619314,
 21888242871839275222246405745257275088548284

In [160]:
# Assuming model_tf is your original model
# Extract the output of the second-to-last layer (the last dense layer before softmax)
layer_output = model_tf.layers[1].output

# Create a new model that gives the output of the last dense layer
intermediate_model = keras.Model(inputs=model_tf.input, outputs=layer_output)

intermediate_model.predict(X.reshape(1, -1)) 

1/1 [==============================] - 0s 22ms/step


array([[5.6443423e-09, 9.9486595e-01, 1.3254236e-03, 4.2857317e-04,
        9.8289172e-07, 8.4703215e-06, 1.3793467e-06, 2.5933378e-03,
        6.8582140e-04, 9.0050235e-05]], dtype=float32)

In [161]:
_, _, _, dense_out_2, remainder = DenseInt(56, 10, 10**36, dense_out_1, weights[1], biases[1], act=False)
dense_out_2

[7735081885698525090671781120107247323615065383037025344961559877598561881588,
 21804873014693066069609384386884114651109037589376155098374850002645650164233,
 17999263188021509643811355194238820250977022962787785447268480961075937305244,
 21324998268924886012428315205051215564239800920986807565515086703270456560467,
 16059188698661913177761974199140049575163288439636430571166107416014577467585,
 14732255293448927243090188448786532403436527249519021875340647289934294884659,
 6821424371073475607027322913928733901969723771150172819978195505822527367491,
 67713646206310789366372044325868952622343505541247674699983513498412536794,
 14433190837606797695021663537720886627321264277759233479515138902368268233524,
 8897477246040641569599502032963516731694805654261979179335584008834406234713]

In [162]:
def mod_exp(x, p):
    return pow(int(x), 1, p)  # Modular exponentiation

def mod_softmax(out, p):
    # Apply modular exponentiation and then normalize
    exp_out = [mod_exp(x, p) for x in out]
    sum_exp_out = sum(exp_out) % p
    softmax_out = [x * pow(sum_exp_out, -1, p) % p for x in exp_out]  # Modular division
    return [int(x) for x in softmax_out]

In [163]:
model_tf.predict(X.reshape(1, -1))  # Reshape if necessary

# 'output' now contains the output of the last dense layer before softmax

1/1 [==============================] - 0s 14ms/step


array([[5.6443423e-09, 9.9486595e-01, 1.3254236e-03, 4.2857317e-04,
        9.8289172e-07, 8.4703215e-06, 1.3793467e-06, 2.5933378e-03,
        6.8582140e-04, 9.0050235e-05]], dtype=float32)

In [164]:
np.argmax(dense_out_2)

1

### Benchmark

In [244]:

weights_ = []
biases_ = []

weights = [[None for _ in range(56)] for _ in range(784)]
for i in range(784):
    for j in range(56):
        weights[i][j] = int(model_tf.get_weights()[0][i][j]*1e36)
        #print (i,j)
biases = [int(b*1e72) for b in model_tf.get_weights()[1]]

weights_.append(weights)
biases_.append(biases)

weights = [[None for _ in range(10)] for _ in range(56)]
for i in range(56):
    for j in range(10):
        weights[i][j] = int(model_tf.get_weights()[2][i][j]*1e36)
        #print (i,j)
biases = [int(b*1e72) for b in model_tf.get_weights()[3]]

weights_.append(weights)
biases_.append(biases)



In [240]:
# Scale the weights and biases
weights = []
biases= []

for layer in model_tf.layers:
    #print ('new layer')
    weight, bias = layer.get_weights()
    scaled_weights = (weight * 1e36).astype(int)
    scaled_biases = (bias * 1e72).astype(int)
    weights.append(scaled_weights)
    biases.append(scaled_biases)

# using relu for dense layer is not supported by keras2circom
def relu_mod(x, p):
    return x if x < p // 2 else 0

/tmp/ipykernel_766371/3291663682.py:8: RuntimeWarning: invalid value encountered in cast
  scaled_weights = (weight * 1e36).astype(int)
/tmp/ipykernel_766371/3291663682.py:9: RuntimeWarning: invalid value encountered in cast
  scaled_biases = (bias * 1e72).astype(int)


In [236]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias, act = False):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]

    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += int(input[i] * weights[i][j])
        out[j] += int(bias[j])

        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p

    if act:
        out = [int(relu_mod(out[i], p)) for i in range(nOutputs)]

    return Input, Weights, Bias, out, remainder    

In [266]:
def input_dense_dense(nInputs, nHidden, nOutputs, X_in, weights, biases):
    X_in, dense_42_weights, dense_42_bias, dense_42_out, dense_42_remainder = DenseInt(nInputs, nHidden, 10**36, X_in, weights[0], biases[0], act=True)
    #print (dense_42_weights)
    _, dense_43_weights, dense_43_bias, dense_43_out, dense_43_remainder = DenseInt(nHidden, nOutputs, 10**36, dense_42_out, weights[1], biases[1], act=True)

    pred = np.argmax(dense_43_out)
    in_json = {
        "in": X_in,
        "dense_42_weights": dense_42_weights,    
        "dense_42_bias": dense_42_bias,     
        "dense_42_out": dense_42_out,    
        "dense_42_remainder": dense_42_remainder,     
        "dense_43_weights": dense_43_weights,     
        "dense_43_bias": dense_43_bias,
        "dense_43_out": dense_43_out,
        "dense_43_remainder": dense_43_remainder,
        #"dense_43_softmax_out": int(pred)
    }

    return in_json, pred

in_json, out = input_dense_dense(784, 56, 10, X_in, weights_, biases_)

In [267]:
out

4

In [268]:
import json
with open("model_input.json", "w") as f:
    json.dump(in_json, f)

: 

In [254]:
def benchmark(test_images, scaled_test_images, model, weights, biases):
    for i in range(len(test_images)):
        X = test_images[i:i+1]
        pred = np.argmax(model.predict(X), axis = 1)

        X_in = scaled_test_images[i]
        in_json, out = input_dense_dense(784, 56, 10, X_in, weights, biases)
        print(pred[0])
        if pred[0] != out:
            print (f"Index {i} Not match!")


In [255]:
benchmark(test_images_tf[:20], scaled_test_images_tf, model_tf, weights_, biases_)

1/1 [==============================] - 0s 18ms/step
7
1/1 [==============================] - 0s 10ms/step
2
1/1 [==============================] - 0s 9ms/step
1
1/1 [==============================] - 0s 9ms/step
0
1/1 [==============================] - 0s 9ms/step
4
1/1 [==============================] - 0s 9ms/step
1
1/1 [==============================] - 0s 9ms/step
4
1/1 [==============================] - 0s 9ms/step
9
1/1 [==============================] - 0s 10ms/step
6
1/1 [==============================] - 0s 9ms/step
9
Index 9 Not match!
1/1 [==============================] - 0s 9ms/step
0
Index 10 Not match!
1/1 [==============================] - 0s 9ms/step
6
1/1 [==============================] - 0s 9ms/step
9
1/1 [==============================] - 0s 10ms/step
0
1/1 [==============================] - 0s 9ms/step
1
1/1 [==============================] - 0s 9ms/step
5
1/1 [==============================] - 0s 9ms/step
9
Index 16 Not match!
1/1 [==============================]

#### Prepare Test Dataset

In [169]:
import json
size = 100

In [204]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

In [207]:
predicted_labels = [int(pred) for pred in get_predictions_tf(model_tf, test_images_tf[:100])]
with open('y_test.json', 'w+') as f:
    json.dump(predicted_labels, f)

4/4 [==============================] - 0s 1ms/step


In [189]:
test_images_ = test_images.astype('float32')
test_images_ /= 255.0

In [208]:
test_folder = "./x_test/"
os.makedirs(test_folder, exist_ok=True)

for i in range(size):
    X_in = [str(int(x * float(10 ** 36))) for x in test_images_[i].flatten().tolist()]
    X_in = np.array(X_in).reshape(28,28,1).tolist()

    #print (X_in)
    with open(f'{test_folder}{i}.json', 'w+') as f:
        json.dump({"in": X_in}, f)


In [211]:
with open (f"./{circuit_folder}/circuit.json") as f:
    circuit = json.load(f)

with open ("y_test.json") as f:
    y_test = json.load(f)

In [212]:
from circuits.circuit import inference
correct = 0
for i in range(size):
    with open(f"{test_folder}{i}.json", "r") as f:
        input = json.load(f)
    
    out, _ = inference(input, circuit)
    correct += 1 if out[0] == y_test[i] else 0
    print ("correct?", correct)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [78]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "remainder": remainder
}

NameError: name 'out' is not defined

In [12]:
with open("dense_input.json", "w") as f:
    json.dump(in_json, f)